In [1]:
import pandas as pd # deal with data and one-hot encoding
import numpy as np # data manipulation
import matplotlib.pyplot as plt #graph
import matplotlib.colors as colors
from sklearn.utils import resample # downsample the dataset
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler,scale # scale and center the data
from sklearn.svm import SVC # model
from sklearn.model_selection import GridSearchCV # cross validation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.decomposition import PCA # to perform PCA to plot the data

import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics

In [2]:
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [3]:
df = pd.read_csv("weatherAUS.csv",header=0)
df = df.dropna().reset_index(drop=True)

In [4]:
df.drop(['Date','Location'],axis=1, inplace=True)
# df.drop(['Date'],axis=1, inplace=True)

cat_cols = ['WindDir9am', 'WindDir3pm', 'WindGustDir']
for col in cat_cols:
    df.loc[(df[col] == 'SSW')|(df[col] == 'SSE'), col] = 'S'
    df.loc[(df[col] == 'NNE')|(df[col] == 'NNW'), col] = 'N'
    df.loc[(df[col] == 'WSW')|(df[col] == 'WNW'), col] = 'W'
    df.loc[(df[col] == 'ENE')|(df[col] == 'ESE'), col] = 'E'

In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
# Check the labels
label_encoder.fit(df['RainTomorrow'])
label_encoder.classes_ 

# transform
df['RainTomorrow'] = label_encoder.transform(df.RainTomorrow)

In [6]:
cat_vars = ['WindDir9am', 'WindDir3pm', 'WindGustDir', 'RainToday']

for var in cat_vars:
    cat_list = pd.get_dummies(df[var], prefix=var)
    df = df.join(cat_list)

In [7]:
## DROP the Old Cat vars: 
df_final = df.drop(cat_vars, axis=1)

In [8]:
df_final.columns

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'RainTomorrow', 'WindDir9am_E', 'WindDir9am_N',
       'WindDir9am_NE', 'WindDir9am_NW', 'WindDir9am_S', 'WindDir9am_SE',
       'WindDir9am_SW', 'WindDir9am_W', 'WindDir3pm_E', 'WindDir3pm_N',
       'WindDir3pm_NE', 'WindDir3pm_NW', 'WindDir3pm_S', 'WindDir3pm_SE',
       'WindDir3pm_SW', 'WindDir3pm_W', 'WindGustDir_E', 'WindGustDir_N',
       'WindGustDir_NE', 'WindGustDir_NW', 'WindGustDir_S', 'WindGustDir_SE',
       'WindGustDir_SW', 'WindGustDir_W', 'RainToday_No', 'RainToday_Yes'],
      dtype='object')

In [9]:
df_final.columns[16]

'RainTomorrow'

In [10]:
X, y = df_final.values[:, :16], df_final.values[:, 16]

In [11]:
# Split dataset into training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=1) # 80% training and 20% test

In [12]:
tree = DecisionTreeClassifier()
model = tree.fit(X_train, Y_train)
y_pred = model.predict(X_test)

In [13]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, y_pred))

Accuracy: 0.7968805388160227


In [18]:
list(zip(df_final.iloc[:, :16].columns.tolist(),model.feature_importances_))

AttributeError: 'SVC' object has no attribute 'feature_importances_'

In [15]:
X_scaler = StandardScaler().fit(X_train)
# y_scaler = StandardScaler().fit(y_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(random_state=42, C=100,gamma=0.0001)))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train_scaled, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.854307 (0.005119)
LDA: 0.852202 (0.005387)
KNN: 0.842188 (0.004989)
CART: 0.797390 (0.004216)
NB: 0.803327 (0.005333)
SVM: 0.854240 (0.004110)


In [ ]:
X, y = df_final[['Humidity3pm','Sunshine','Pressure3pm','Pressure9am']].values, df_final.values[:, -27]
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=1) # 80% training and 20% test

In [ ]:
tree = DecisionTreeClassifier()
model = tree.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))